In [ ]:
import autosklearn.classification
import autosklearn.regression
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import numpy as np

# Пример классификации
# --------------------
# Загрузка данных
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.25, random_state=42
)

# Настройка и запуск Auto-Sklearn для классификации
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,  # время в секундах (1 час)
    per_run_time_limit=360,        # ограничение времени для одной модели (6 минут)
    n_jobs=-1,                     # использовать все ядра
    memory_limit=8192,             # ограничение памяти в МБ (8 ГБ)
    tmp_folder='./auto_sklearn_tmp',
    ensemble_size=50,              # размер ансамбля
    initial_configurations_via_metalearning=25,  # количество начальных конфигураций
    resampling_strategy='cv',       # стратегия ресемплинга
    resampling_strategy_arguments={'folds': 5}  # параметры ресемплинга
)

# Обучение
automl.fit(X_train, y_train)

# Получение лучших моделей
print(automl.leaderboard())
print(automl.show_models())

# Вычисление метрик на тестовой выборке
y_pred = automl.predict(X_test)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred)}")
print(f"Balanced Accuracy: {sklearn.metrics.balanced_accuracy_score(y_test, y_pred)}")
print(f"F1: {sklearn.metrics.f1_score(y_test, y_pred)}")
print(f"AUC: {sklearn.metrics.roc_auc_score(y_test, y_pred)}")

# Вероятности классов
y_pred_proba = automl.predict_proba(X_test)

# Для регрессии
# ------------
X, y = sklearn.datasets.fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.25, random_state=42
)

automl_reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=3600,
    per_run_time_limit=360,
    n_jobs=-1,
    memory_limit=8192
)

automl_reg.fit(X_train, y_train)

# Оценка модели
y_pred = automl_reg.predict(X_test)
print(f"R2: {sklearn.metrics.r2_score(y_test, y_pred)}")
print(f"RMSE: {np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred))}")
print(f"MAE: {sklearn.metrics.mean_absolute_error(y_test, y_pred)}")

In [ ]:
automl = autosklearn.classification.AutoSklearnClassifier(
    include_estimators=["random_forest", "adaboost", "xgradient_boosting"],  # включить только эти
    exclude_estimators=["gaussian_nb", "libsvm_svc"],  # исключить эти
    include_preprocessors=["pca", "feature_agglomeration"],  # включить предобработку
    exclude_preprocessors=["kitchen_sinks", "nystroem_sampler"]  # исключить предобработку
)

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Пример данных с категориальными признаками
data = pd.read_csv('data_with_categorical.csv')
X = data.drop('target', axis=1)
y = data['target']

# Разделение признаков на числовые и категориальные
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Создание трансформера для преобразования данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Преобразование данных
X_processed = preprocessor.fit_transform(X)

# Далее используем Auto-Sklearn
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X_processed, y, test_size=0.25, random_state=42
)

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,
    per_run_time_limit=360
)
automl.fit(X_train, y_train)

In [ ]:
# Построение ансамбля из моделей, выбранных Auto-Sklearn
from autosklearn.ensembles.ensemble_selection import EnsembleSelection

# После выполнения automl.fit()
# Получаем список моделей
models = automl.get_models_with_weights()

# Создаем свой ансамбль
ensemble = EnsembleSelection(
    ensemble_size=10,
    task_type=1,  # 1 для классификации, 2 для регрессии
    metric=autosklearn.metrics.accuracy
)

# Добавляем модели в ансамбль и их веса
for model, weight in models:
    ensemble.append(model, weight)

# Предсказание с использованием ансамбля
y_pred = ensemble.predict(X_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

# После выполнения automl.fit()
best_model = automl.get_models_with_weights()[0][0]

# Создаем пайплайн с PCA и лучшей моделью
pipeline = Pipeline([
    ('pca', PCA(n_components=10)),
    ('best_model', best_model)
])

# Обучаем пайплайн
pipeline.fit(X_train, y_train)

# Делаем предсказания
y_pred = pipeline.predict(X_test)